In [ ]:
import pygrib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import cartopy.crs as crs
import cartopy.feature as cfeature

In [ ]:
### Define the path and name of the monthly averaged grib file and open using pygrib.
filename = 'ERA5MonthlyAveraged.grib'
grbs = pygrib.open(filename) 
grb = grbs.select()[:]
grb

In [ ]:
###Declare some empty lists for each variable
u     = []
v     = []
waves = []

###Loop over the grib messages and append the above lists based on the grib message .name string
for i in np.arange(0,len(grb)):
    if 'U wind' in grb[i].name:
        data, ulats, ulons = grb[i].data() ### Get the data, lats, and lons from the grib message
        u.append(data)
    if 'V wind' in grb[i].name:
        data, vlats, vlons = grb[i].data()
        v.append(data)
    if 'waves' in grb[i].name:
        data, wavelats, wavelons = grb[i].data()
        waves.append(data)

In [ ]:
###Check the dimensions of the lists we filled
print('The length of the u list is: ',len(u))
print('The shape of the data in each year is: ',u[0].shape)

In [ ]:
###Get the decadal mean by using np.mean along axis 0 (the year dimension)
udecade = np.mean(u,axis=0)
vdecade = np.mean(v,axis=0)
spddecade = ((udecade**2)+(vdecade**2))**(0.5) #Use Pythagorean Theorem to get wind speed
wavesdecade = np.mean(waves,axis=0)

In [ ]:
###Make a quick plot of the wind speed
windlevs = np.arange(0,4.5,0.5) #declare some contour levels for wind speed
plt.contourf(ulons,ulats,spddecade,levels=windlevs)
plt.quiver(ulons,ulats,udecade,vdecade)
plt.colorbar()

In [ ]:
###Make a quick plot of the waves
wavelevs = np.arange(0,2,0.25) #declare some contour levels for the wave height
plt.contourf(wavelons,wavelats,wavesdecade,levels=wavelevs)
plt.colorbar()

In [ ]:
# Create a figure and axis with a specific projection
fig, ax = plt.subplots(figsize=(12, 6), subplot_kw={'projection': crs.PlateCarree()})

# Create a contour plot
windlevs = np.arange(0,4.5,0.5)
contour = ax.contourf(ulons,ulats,spddecade,levels=windlevs, transform=crs.PlateCarree())
# Create a quiver plot for the wind vectors
Q = ax.quiver(ulons[::2,::2],ulats[::2,::2],udecade[::2,::2],vdecade[::2,::2],scale=80) #Plot wind vectors at every other grid point
qk = ax.quiverkey(Q, 0.1, -0.1, 5, r'$5 \frac{m}{s}$', labelpos='W') #This provides the reference vector (bottom left)

# Add coastlines and state borders
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.STATES)

# Add a colorbar
cbar = plt.colorbar(contour)
cbar.set_label('Wind speed (m/s)')

# Set title
plt.title('Decadal mean wind speed (m/s) for November from 1965-1975')

# Show the plot
plt.show()

In [ ]:
# Create a figure and axis with a specific projection
fig, ax = plt.subplots(figsize=(12, 6), subplot_kw={'projection': crs.PlateCarree()})

# Create a contour plot
wavelevs = np.arange(0,2,0.25)
contour = ax.contourf(wavelons,wavelats,wavesdecade,levels=wavelevs, transform=crs.PlateCarree())

# Add coastlines and state borders
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.STATES)

# Add a colorbar
cbar = plt.colorbar(contour)
cbar.set_label('Wave height (m)')

# Set title
plt.title('Decadal mean wave height (m) for November from 1965-1975')

# Show the plot
plt.show()

# Now get the anomalies

In [ ]:
###Define the path and name of the (six) hourly grib file and open using pygrib.
filename = 'ERA5Hourly.grib'
grbs = pygrib.open(filename) 
grb = grbs.select()[:]
grb

In [ ]:
###Declare some empty lists for each variable
hu     = []
hv     = []
hwaves = []

###Loop over the grib messages and append the above lists based on the grib message .name string

for i in np.arange(0,len(grb)):
    if 'U wind' in grb[i].name:
        data, ulats, ulons = grb[i].data() ### Get the data, lats, and lons from the grib message
        hu.append(data)
    if 'V wind' in grb[i].name:
        data, vlats, vlons = grb[i].data()
        hv.append(data)
    if 'waves' in grb[i].name:
        data, wavelats, wavelons = grb[i].data()
        hwaves.append(data)

In [ ]:
### Get the anomalies at 00z on Nov 11 (index 4 in the lists), around the time that the Edmund Fitzgerald sank
anom00z_u = hu[4]-udecade
anom00z_v = hv[4]-vdecade
spd00z = ((hu[4]**2)+(hv[4]**2))**(0.5)
anom00z_spd = spd00z-spddecade
anom00z_waves = hwaves[4]-wavesdecade

In [ ]:
# Create a figure and axis with a specific projection
fig, ax = plt.subplots(figsize=(12, 6), subplot_kw={'projection': crs.PlateCarree()})

# Create a contour plot
windlevs = np.arange(0,24,2)
contour = ax.contourf(ulons,ulats,anom00z_spd,levels=windlevs, transform=crs.PlateCarree())
Q = ax.quiver(ulons[::2,::2],ulats[::2,::2],anom00z_u[::2,::2],anom00z_v[::2,::2],scale=300) #Plot wind vectors at every other grid point
qk = ax.quiverkey(Q, 0.1, -0.1, 10, r'$10 \frac{m}{s}$', labelpos='W')

# Plot the location of the wreck of The Edmund Fitzgerald
wrecklat  = 46.9985
wrecklon  = -85.1102
wreckloc  = ax.scatter(wrecklon,wrecklat,color='red',marker='x')

# Add coastlines and state borders
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.STATES)

# Add a colorbar
cbar = plt.colorbar(contour)
cbar.set_label('Wind speed (m/s)')

# Set title
plt.title('Wind speed anomalies (m/s) at 00z on Nov. 11, 1975')

# Show the plot
plt.show()

In [ ]:
# Create a figure and axis with a specific projection
fig, ax = plt.subplots(figsize=(12, 6), subplot_kw={'projection': crs.PlateCarree()})

# Create a contour plot
wavelevs = np.arange(0,7.5,0.5)
contour = ax.contourf(wavelons,wavelats,anom00z_waves,levels=wavelevs, transform=crs.PlateCarree())

# Plot the location of the wreck of The Edmund Fitzgerald
wrecklat  = 46.9985
wrecklon  = -85.1102
wreckloc  = ax.scatter(wrecklon,wrecklat,color='red',marker='x')

# Add coastlines and state borders
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.STATES)

# Add a colorbar
cbar = plt.colorbar(contour)
cbar.set_label('Wave height (m)')

# Set title
plt.title('Wave height anomalies (m) at 00z on Nov. 11, 1975')

# Show the plot
plt.show()

# Interpretation?
During the wreck, winds were ... 

During the wreck, waves were ...

Based on the anomalous wind pattern it looks like...